In [1]:
%config IPCompleter.evaluation='unsafe'

In [2]:
from pyiron_atomistics import Project
from atomrdf import KnowledgeGraph
import numpy as np

In [3]:
project = 'y8'
pr = Project(project)
kg = KnowledgeGraph(store='db', store_file=f'{project}.db')
kg.enable_workflow(pr, workflow_environment='pyiron')

In [4]:
structure = pr.create.structure.annotated_structure.bulk('Cu', cubic=True, label='cu md', repetitions=(3, 3, 3))
job = pr.create.job.Lammps('bulk', delete_existing_job=True, delete_aborted_job=True)
job.structure = structure
job.potential = '2001--Mishin-Y--Cu-1--LAMMPS--ipr1'
job.calc_md(pressure=0, temperature=500)
job.run()
kg.add_workflow(job, workflow_environment='pyiron')

The job bulk was saved and received the ID: 1214


In [5]:
kg.samples

[cu_md, sample:b5b503c7-f5a6-4794-a1c1-042c7fa77add_from_cu_md]

In [6]:
sample = kg.samples[1]

In [7]:
min_struct = sample.structure

In [8]:
del min_struct[3]

In [9]:
job = pr.create.job.Lammps('vacancy', delete_existing_job=True, delete_aborted_job=True)
job.structure = min_struct.write.pyiron()
job.potential = '2001--Mishin-Y--Cu-1--LAMMPS--ipr1'
job.calc_static()
job.run()
kg.add_workflow(job, workflow_environment='pyiron')

The job vacancy was saved and received the ID: 1215


form_energy = energy_vac - ((number_atoms-1)/number_atoms)*energy_bulk

In [10]:
kg.samples

[cu_md,
 sample:b5b503c7-f5a6-4794-a1c1-042c7fa77add_from_cu_md,
 sample:7f0eed16-820c-4ff1-baa0-64cf2f08f027_from_sample:b5b503c7-f5a6-4794-a1c1-042c7fa77add_from_cu_md]

In [11]:
vac_sample = kg.samples[2]
bulk_sample = kg.samples[1]

In [12]:
e_form = vac_sample.outputs.TotalEnergy - ((bulk_sample.properties.NumberOfAtoms - 1)/bulk_sample.properties.NumberOfAtoms)*bulk_sample.outputs.TotalEnergy

In [13]:
e_form.value

-5.688631775700969

In [14]:
e_form.label = 'VacancyFormationEnergy'

In [15]:
kg.generate_provenance(label='VacancyFormationEnergy', visualize=False)

http://purls.helmholtz-metadaten.de/asmo/wasCalculatedBy operation:ba8d62e4-8049-4d1f-b596-4d2713b91d71
http://purls.helmholtz-metadaten.de/asmo/wasCalculatedBy


{'property:34a2146f-25c5-4498-aab9-0599937098e2': {'found': True,
  'label': 'VacancyFormationEnergy',
  'operation': 'output_parameter',
  'inputs': {'0': 'operation:ba8d62e4-8049-4d1f-b596-4d2713b91d71'}},
 'operation:ba8d62e4-8049-4d1f-b596-4d2713b91d71': {'found': True,
  'label': 'operation:ba8d62e4-8049-4d1f-b596-4d2713b91d71',
  'inputs': {},
  'operation': 'sample_for_activity'}}

In [19]:
from rdflib import URIRef

In [21]:
for x in kg.triples((URIRef('operation:412b11a7-0bfd-48d9-addb-851e244dbadf'), None, None)):
    print(x)

(rdflib.term.URIRef('operation:412b11a7-0bfd-48d9-addb-851e244dbadf'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://purls.helmholtz-metadaten.de/asmo/Subtraction'))
(rdflib.term.URIRef('operation:412b11a7-0bfd-48d9-addb-851e244dbadf'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/ns/prov#Activity'))
(rdflib.term.URIRef('operation:412b11a7-0bfd-48d9-addb-851e244dbadf'), rdflib.term.URIRef('http://purls.helmholtz-metadaten.de/asmo/hasMinuend'), rdflib.term.URIRef('activity:b1d6751d-16b6-4ae8-9ace-55da70f803db_TotalEnergy'))
(rdflib.term.URIRef('operation:412b11a7-0bfd-48d9-addb-851e244dbadf'), rdflib.term.URIRef('http://purls.helmholtz-metadaten.de/asmo/hasSubtrahend'), rdflib.term.URIRef('property:7c88abd8-93f0-450a-8739-446e78a3e565'))
(rdflib.term.URIRef('operation:412b11a7-0bfd-48d9-addb-851e244dbadf'), rdflib.term.URIRef('http://purls.helmholtz-metadaten.de/asmo/hasDi